# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397703008689                   -0.90    4.8   25.0ms
  2   -8.400205046229       -2.60       -1.74    1.0   18.6ms
  3   -8.400407222918       -3.69       -2.96    1.8   50.0ms
  4   -8.400427897730       -4.68       -3.00    3.0   23.4ms
  5   -8.400427976603       -7.10       -3.08    1.0   18.6ms
  6   -8.400428146613       -6.77       -4.71    1.0   18.6ms
  7   -8.400428151859       -8.28       -4.48    3.0   23.5ms
  8   -8.400428152183       -9.49       -5.12    1.0   18.8ms
  9   -8.400428152207      -10.62       -6.47    1.0   41.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397773248235                   -0.90           5.0    826ms
  2   -8.400376448887       -2.58       -1.77   0.80    2.2    508ms
  3   -8.400422276893       -4.34       -2.98   0.80    1.0    186ms
  4   -8.400428107238       -5.23       -3.40   0.80    2.5   20.2ms
  5   -8.400428148832       -7.38       -4.69   0.80    1.2   16.9ms
  6   -8.400428152181       -8.48       -5.49   0.80    2.8   20.5ms
  7   -8.400428152208      -10.56       -6.18   0.80    2.2   58.6ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.451564217883                   -1.07    2.42s
  2   -1.670673739798        0.49       -0.68    120ms
  3   -4.197987194022        0.40       -0.42   43.3ms
  4   -5.945974697708        0.24       -0.57   84.6ms
  5   -7.404908828107        0.16       -0.72   43.7ms
  6   -7.700079133346       -0.53       -1.33   32.4ms
  7   -7.985053480536       -0.55       -1.44   32.3ms
  8   -7.985935699004       -3.05       -1.78   56.8ms
  9   -8.182191849185       -0.71       -1.46   32.4ms
 10   -8.304583172656       -0.91       -1.54   43

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397867962757                   -0.90    5.2   26.1ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427979196                   -1.78    24.2s
  2   -8.400428152209       -6.76       -4.02    1.95s
  3   -8.400428152209      -14.45       -7.82    101ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.05885444763951e-7
|ρ_newton - ρ_scfv| = 3.4395084441889135e-7
|ρ_newton - ρ_dm|   = 2.810614849705745e-6
